### Install dependencies

In [ ]:
pip install pandas numpy matplotlib

### Initialize ODTK

In [ ]:
import subprocess
odtkProcess = subprocess.Popen(['odtkruntime'], stderr=subprocess.PIPE)
while True:
    if b'ODTK initialized' in odtkProcess.stderr.readline():
        break

### Access ODTK API

In [ ]:
from odtk import Client
client = Client()
odtk = client.get_root()
odtk.application.appVersion.eval()

### Create New Objects

In [ ]:
odtk.application.deleteObject('', odtk.scenario[0])
scenario = odtk.application.createObj(odtk, 'Scenario', 'TestScenario')
odtk.application.createObj(scenario, 'Satellite', 'TestSatellite')
trackingSystem = odtk.application.LoadObj(scenario, '/home/odtk/ODTK/AppData/Databases/TrackingSystems/Baseline.tso')

### Simulate Observations

In [ ]:
simulator = odtk.application.createObj(scenario, 'Simulator', 'TestSimulator')
simulator.Go()

### Run Filter

In [ ]:
filter = odtk.application.createObj(scenario, 'Filter', 'TestFilter')
filter.Go()

### Export Data

In [ ]:
odtk.ProductBuilder.DataProducts.Clear()
input_file_name = filter.Output.DataArchive.Filename.value.eval()
style_file_path = "/home/odtk/data/Styles/ResidualRatios.exp"
output_file_path = "/home/odtk/.ODTK7/DataArchive/Residuals.csv"

new_elem = odtk.ProductBuilder.DataProducts.NewElem()
odtk.ProductBuilder.DataProducts.push_back(new_elem)
index = odtk.ProductBuilder.DataProducts.size.eval() - 1
product = odtk.ProductBuilder.DataProducts[index]

new_src = product.Inputs.DataSources.NewElem()
product.Inputs.DataSources.push_back(new_src)
product.Name.Assign('residuals')
product.Inputs.DataSources[0].Filename = input_file_name
product.Outputs.Style = style_file_path
product.Outputs.Display = False
product.Outputs.Export.Enabled = True
product.Outputs.Export.Format = "CommaSeparated"
product.Outputs.Export.FileName = output_file_path

if not odtk.ProductBuilder.GenerateProduct('residuals'):
    raise Exception('GenerateProduct failed. Please check the log for more details.')
    

### Plot Residuals

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv(output_file_path, parse_dates=["Date Time"])
gb = data.groupby("Measurement Type")
gb.plot.scatter("Date Time","Residual Ratio", rot=90)

### Clean Up

In [ ]:
odtk.application.deleteObject('', odtk.scenario[0])
odtkProcess.terminate()